In [ ]:
# load test datasets
!cp /content/drive/Shareddrives/RAI/tests/datasets.zip /content
!unzip /content/datasets.zip -d /content

# 1. DeepFace


In [ ]:
# pre install
!pip install deepface

In [ ]:
import cv2
from deepface import DeepFace
import numpy as np
from os import walk

emo = ['neutral', 'happy', 'sad', 'surprise', 'anger', 'fear']
total=0
correct=0
for i in emo:
    filenames = next(walk(f'/content/datasets/{i}/'), (None, None, []))[2]
    for j in filenames:
        try:
            path = f"/content/datasets/{i}/{j}"
            img_array = np.fromfile(path, np.uint8)
            img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
            label = DeepFace.analyze(img,actions=['emotion'])['dominant_emotion']
            if label==i : correct+=1
            total+=1
        except:
            continue
    print(i)

print((correct*100)/total)

facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 91.5MB/s]


neutral
happy
sad
surprise
anger
fear
28.953771289537713


# 2. DAN

In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/RAI/models/DAN/networks/')
from dan import DAN

import os
import argparse
from os import walk

from PIL import Image

import torch
from torchvision import transforms

In [ ]:
class Model():
    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.data_transforms = transforms.Compose([
                                    transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
                                ])
        self.labels = ['neutral', 'happy', 'sad', 'surprise', 'fear', 'disgust', 'anger']

        self.model = DAN(num_head=4, num_class=7, pretrained=False)
        checkpoint = torch.load('/content/drive/Shareddrives/RAI/models/DAN/affecnet7_epoch6_acc0.6569.pth',
            map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'],strict=True)
        self.model.to(self.device)
        self.model.eval()
    
    def fit(self, path):
        img = Image.open(path).convert('RGB')
        img = self.data_transforms(img)
        img = img.view(1,3,224,224)
        img = img.to(self.device)

        with torch.set_grad_enabled(False):
            out, _, _ = self.model(img)
            _, pred = torch.max(out,1)
            index = int(pred)
            label = self.labels[index]

            return index, label

In [ ]:
model = Model()

emo = ['neutral', 'happy', 'sad', 'surprise', 'anger', 'fear']
total=0
correct=0
for i in emo:
  filenames = next(walk(f'/content/datasets/{i}/'), (None, None, []))[2]
  for j in filenames:
      image = f"./datasets/{i}/{j}"

      index, label = model.fit(image)
      if label==i : correct+=1
      total+=1

print((correct*100)/total)

19.53757225433526


# CNN

In [ ]:
!pip install fer

In [ ]:
from fer import FER
import cv2
from os import walk
import numpy as np

In [ ]:
emo = ['neutral', 'happy', 'sad', 'surprise', 'anger', 'fear']
total=0
correct=0
detector = FER()
for i in emo:
    filenames = next(walk(f'./datasets/{i}/'), (None, None, []))[2]
    for j in filenames:
        path = f"./datasets/{i}/{j}"
        img_array = np.fromfile(path, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        label, score = detector.top_emotion(img)
        if label==i : correct+=1
        total+=1
    print(i)

print((correct*100)/total)

neutral
happy
sad
surprise
anger
fear
38.03468208092485
